
# Incorporating OLI Calculations with WaterTAP

#### Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)

This tutorial will demonstrate basic usage of OLI Cloud calls using our custom API tools.

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## Required OLI API Inputs


 - State variables (solute concentrations, temperature, pressure), which can beextracted from a state block
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - establishes state variables, phases, etc. to be considered in flash calls

In [ ]:
# used to specify state/convert variables
from pyomo.environ import units as pyunits

# used to build survey lists
from numpy import linspace

# used to execute OLI Cloud functions
from watertap.tools.oli_api.flash import Flash
from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

# 1. Specify State Variables.

- This data is used to construct inputs to OLI Cloud
- Same basic information is required by Water Analysis and Isothermal flashes (temperature, pressure, solutes)

In [ ]:
state_vars = {
    "temperature": 298.15,
    "pressure": 101325,
    "components": {
        "Na_+": 1e3,
        "Cl_-": 1e3
    },
    "units": {
        "temperature": pyunits.K,
        "pressure": pyunits.Pa,
        "components": pyunits.mg / pyunits.L,
    },
}

# 2. Initialize Flash Instance.

 - We will run most of our methods with this class

In [ ]:
f = Flash()

# 3. Get Survey Parameters.

 - In this example, we will generate a temperature sweep survey

In [ ]:
# a survey will sweep through one or more variables simultaneously
survey = f.build_survey(
    {"Na_+": linspace(0, 1e5, 10)},
    get_oli_names=True,
    file_name="test_survey",
)

In [ ]:
# individual sample points can be accessed to see what will be modified
samples = range(0,10,2)
survey_points = f.get_survey_sample_conditions(survey, samples)
f.write_output(survey_points, "test_survey_points")

# 4. Login to OLI Cloud.

- The following code demonstrates an OLI Cloud login:

In [ ]:
try:
    # credentials are entered here
    # up to 5 access keys per user can be generated
    # CredentialManager can save login information and access keys
    credential_manager = CredentialManager(
        username="", # requires password, root_url, and auth_url
        password="", # required with username login only
        root_url="", # required with username and access key login
        auth_url="", # required with username login only
        access_keys=[], # only requires root_url
    )
except (OSError, ConnectionError) as e:
    print(e)
    credential_manager = None

# 5. Create *.dbs File and 6. Get Output

In [ ]:
if credential_manager:
    # OLIApi is used as a context manager
    with OLIApi(credential_manager) as oliapi:
        # create a new DBS file
        # alternative thermo_frameworks and private_databanks are available
        dbs_file_id = oliapi.get_dbs_file_id(
            state_vars["components"],
            thermo_framework="MSE (H3O+ ion)",
            private_databanks=["XSC"],
            phases=["liquid1", "solid"], # liquid2 (non-aqueous) and vapor phases can also be investigated
            model_name="test",
        )
        
        # get a summary of the DBS file
        file_summary = oliapi.get_dbs_file_summary(dbs_file_id)
        
        # save chemistry information
        chemistry_info = file_summary["chemistry_info"]
        f.write_output(chemistry_info["result"], "chemistry_info")
        
        # create water analysis input
        # Water Analysis uses true species for inputs, i.e., IONS
        stream_input = f.build_flash_calculation_input(
            "wateranalysis",
            state_vars,
            file_name="test_water_analysis_input",
        )    
        # run Water Analysis flash calculation survey as specified        
        stream_output = f.run_flash(
            "wateranalysis",
            oliapi,
            dbs_file_id,
            stream_input,
            survey,
            file_name="test_water_analysis_survey_output"
        )
        # The output of Water Analysis gives apparent species
        # i.e., SALTS, COMPLEXES (MgO, CaO, etc.)
        
        # Other flash calculations use apparent species for inputs
        # get_inflows extracts Water Analysis outputs to use with Isothermal flash
        # these apparent species can be saved to a file and reloaded as needed
        isothermal_inflows = f.get_inflows(
            stream_output,
            file_name="isothermal_inflows",
        )
        isothermal_input = f.build_flash_calculation_input(
            "isothermal",
            isothermal_inflows,
            file_name=f"isothermal_input",
        )
        # specify a new survey to compare apparent species
        isothermal_survey = f.build_survey(
            {"NaCl": linspace(0, 1e6, 10)},
            get_oli_names=True,
            file_name="test_isothermal_survey",
        )
        isothermal_output = f.run_flash(
            "isothermal",
            oliapi,
            dbs_file_id,
            isothermal_inflows,
            isothermal_survey,
        file_name=f"{method}_output",
    )


# 7. Extract Filtered Output

 - OLI's output is robust, so WaterTAP enables printing selected results:

In [ ]:
if credential_manager:
    # extract properties from OLI flash survey output
    properties = [
        "prescalingTendencies",
        "molecularConcentration",
    ]
    # data can be saved as a JSON file for further review and analysis
    water_analysis_extract = f.extract_properties(
        stream_output,
        properties=properties,
        file_name="water_analysis_extract",
    )
    isothermal_extract = f.extract_properties(
    isothermal_output,
    properties=properties,
    file_name="isothermal_extract",
)